In [46]:
import pandas as pd
import sqlite3

In [47]:
#Load the CSV file
huddf = pd.read_csv('data/hud_insured_multifamily.csv')

C:\Users\mdalt\AppData\Local\Temp\ipykernel_34900\3972625611.py:2: DtypeWarning: Columns (2,19,22,35,36,41,89,90,91,92,95,103,109,111,121,123,125,126,127,164,169,170,176,205,209,217,274) have mixed types. Specify dtype option on import or set low_memory=False.
  huddf = pd.read_csv('data/hud_insured_multifamily.csv')


In [48]:
# Data cleaning
huddf = huddf.drop(columns=["PROJECT_MANAGER_NAME_TEXT", "AUTOMATIC_GEOCODE_IND"])
huddf = huddf[["STD_CITY","PROPERTY_ID","PROPERTY_CATEGORY_NAME","PROPERTY_NAME_TEXT"]]
huddf.to_csv('data/hud_insured_multifamily_cle.csv', index=False)


In [49]:
# Connect to a SQLite database
conn = sqlite3.connect('database.db')


In [50]:
# TABLE command
create_table_command = """
CREATE TABLE IF NOT EXISTS HUD_Insured_Multifamily (
    STD_CITY,
    PROPERTY_ID,
    PROPERTY_CATEGORY_NAME,
    Property_NAME_TEXT 
)
"""
conn.execute(create_table_command)
conn.commit()

In [51]:
# Function to insert values from CSV to the table
def insert_values_from_csv(conn, csv_file_path):
    df = pd.read_csv(csv_file_path)
    df.to_sql('HUD_Insured_Multifamily', conn, if_exists='replace', index=False)


In [52]:
# Insert values into the table from a CSV file
insert_values_from_csv(conn, 'data/hud_insured_multifamily_cle.csv')

In [53]:
# Query the table and print the result
table = pd.read_sql_query("SELECT * FROM HUD_Insured_Multifamily", conn)
print(table)

                           STD_CITY  PROPERTY_ID PROPERTY_CATEGORY_NAME  \
0      Pennington                    800225589.0   Insured-Unsubsidized   
1      White Settlement              800225933.0   Insured-Unsubsidized   
2      Carlinville                   800225983.0   Insured-Unsubsidized   
3      Grand Coulee                  800226573.0   Insured-Unsubsidized   
4      Vidalia                       800226611.0   Insured-Unsubsidized   
...                             ...          ...                    ...   
16519  Maplewood                     800254374.0   Insured-Unsubsidized   
16520  Garland                       800255132.0   Insured-Unsubsidized   
16521  Pleasant Hill                 800255134.0   Insured-Unsubsidized   
16522  Grasonville                   800255144.0   Insured-Unsubsidized   
16523  Southgate                     800255510.0   Insured-Unsubsidized   

                               PROPERTY_NAME_TEXT  
0                           Capital Health Syst

In [54]:
# Prepare for answering the questions
answers = []

In [55]:
# What city has the most properties?
cursor = conn.execute("SELECT STD_CITY, COUNT(*) FROM HUD_Insured_Multifamily GROUP BY STD_CITY ORDER BY COUNT(*) DESC LIMIT 1")
answers.append(f"City with most properties: {cursor.fetchone()}")


In [56]:
# How many property names start with the letter 'H'
cursor = conn.execute("SELECT COUNT(*) FROM HUD_Insured_Multifamily WHERE Property_NAME_TEXT LIKE 'H%'")
answers.append(f"Properties starting with 'H': {cursor.fetchone()[0]}")


In [57]:
# What % of the properties in the property category is 'Insured-Unsubsidized'
cursor = conn.execute("SELECT COUNT(*) FROM HUD_Insured_Multifamily WHERE PROPERTY_CATEGORY_NAME = 'Insured-Unsubsidized'")
total_count = cursor.fetchone()[0]
cursor = conn.execute("SELECT COUNT(*) FROM HUD_Insured_Multifamily")
overall_count = cursor.fetchone()[0]
percentage = (total_count / overall_count) * 100 if overall_count > 0 else 0
answers.append(f"Percentage of 'Insured-Unsubsidized': {percentage:.2f}%")

In [58]:
# How many unique property id are there in this data set
cursor = conn.execute("SELECT COUNT(DISTINCT PROPERTY_ID) FROM HUD_Insured_Multifamily")
answers.append(f"Unique property IDs: {cursor.fetchone()[0]}")

In [59]:
# Close the connection
conn.close()

In [60]:
# Combine questions and answers into a DataFrame
qa_df = pd.DataFrame({
    "Questions": [
        "What city has the most properties?",
        "How many property names start with the letter 'H'",
        "What % of the properties in the property category is 'Insured-Unsubsidized'",
        "How many unique property IDs are there in this data set"
    ],
    "Answers": answers
})

In [61]:
# Export the DataFrame to a CSV file
qa_df.to_csv(r'data/Questions_and_Answers.csv', index=False)

In [62]:
print(qa_df)

                                           Questions  \
0                 What city has the most properties?   
1  How many property names start with the letter 'H'   
2  What % of the properties in the property categ...   
3  How many unique property IDs are there in this...   

                                             Answers  
0  City with most properties: ('Chicago          ...  
1                  Properties starting with 'H': 801  
2       Percentage of 'Insured-Unsubsidized': 68.70%  
3                         Unique property IDs: 16523  
